In [0]:
!pip install Faker

Requirement already satisfied: Faker in /databricks/python3/lib/python3.8/site-packages (13.12.1)
Requirement already satisfied: python-dateutil>=2.4 in /databricks/python3/lib/python3.8/site-packages (from Faker) (2.8.1)
Requirement already satisfied: six>=1.5 in /databricks/python3/lib/python3.8/site-packages (from python-dateutil>=2.4->Faker) (1.15.0)
 WARNING: You are using pip version 21.0.1; however, version 22.1.2 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
from faker import Faker

faker = Faker()


In [0]:
allempData = ((faker.name(), "Sales", 4000), \
    (faker.name(), "Sales", 5000),  \
    (faker.name(), "Sales", faker.random_int(5000, 10000)),   \
    (faker.name(), "Sales", 5000),    \
    (faker.name(), "Sales", faker.random_int(5000, 10000)), \
    (faker.name(), "Finance", 3000),  \
    (faker.name(), "Finance", 4500),  \
    (faker.name(), "Finance", 7500),    \
    (faker.name(), "IT", faker.random_int(5000, 10000)),\
    (faker.name(), "IT", faker.random_int(5000, 10000)),\
    (faker.name(), "Marketing", 5500), \
    (faker.name(), "Marketing", 5500),\
    (faker.name(), "Marketing", faker.random_int(5000, 10000))\
  )
 
columns= ["EmployeeName", "Department", "Salary"]
df = spark.createDataFrame(data = allempData, schema = columns)
df.show()




+---------------+----------+------+
 EmployeeName|Department|Salary|
+---------------+----------+------+
 John Green| Sales| 4000|
Stephanie Poole| Sales| 5000|
 Erica Rush| Sales| 6731|
 Andrea Scott| Sales| 5000|
 David White| Sales| 5601|
 Matthew Kelly| Finance| 3000|
 Andrea Lawson| Finance| 4500|
 Emily Wells| Finance| 7500|
 Terry Stokes| IT| 7400|
 Todd Stone| IT| 6885|
 Brian Chavez| Marketing| 5500|
 Hannah Weaver| Marketing| 5500|
 Dawn Aguirre| Marketing| 8320|
+---------------+----------+------+

In [0]:
df.printSchema()

root
-- EmployeeName: string (nullable = true)
-- Department: string (nullable = true)
-- Salary: long (nullable = true)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number,rank,dense_rank,percent_rank,ntile,lead,lag,col,sum,avg,max

windowSpec  = Window.partitionBy("Department").orderBy("Salary")
windowSpec_NOB  = Window.partitionBy("Department")

df.withColumn("row_number",row_number().over(windowSpec)) \
    .show()

+---------------+----------+------+----------+
 EmployeeName|Department|Salary|row_number|
+---------------+----------+------+----------+
 Matthew Kelly| Finance| 3000| 1|
 Andrea Lawson| Finance| 4500| 2|
 Emily Wells| Finance| 7500| 3|
 Todd Stone| IT| 6885| 1|
 Terry Stokes| IT| 7400| 2|
 Brian Chavez| Marketing| 5500| 1|
 Hannah Weaver| Marketing| 5500| 2|
 Dawn Aguirre| Marketing| 8320| 3|
 John Green| Sales| 4000| 1|
 Andrea Scott| Sales| 5000| 2|
Stephanie Poole| Sales| 5000| 3|
 David White| Sales| 5601| 4|
 Erica Rush| Sales| 6731| 5|
+---------------+----------+------+----------+

In [0]:
df.withColumn("rank",rank().over(windowSpec)).\
withColumn("dense_rank",dense_rank().over(windowSpec)).filter(col("dense_rank")<=2).show()

+---------------+----------+------+----+----------+
 EmployeeName|Department|Salary|rank|dense_rank|
+---------------+----------+------+----+----------+
 Matthew Kelly| Finance| 3000| 1| 1|
 Andrea Lawson| Finance| 4500| 2| 2|
 Todd Stone| IT| 6885| 1| 1|
 Terry Stokes| IT| 7400| 2| 2|
 Hannah Weaver| Marketing| 5500| 1| 1|
 Brian Chavez| Marketing| 5500| 1| 1|
 Dawn Aguirre| Marketing| 8320| 3| 2|
 John Green| Sales| 4000| 1| 1|
Stephanie Poole| Sales| 5000| 2| 2|
 Andrea Scott| Sales| 5000| 2| 2|
+---------------+----------+------+----+----------+

In [0]:
df.withColumn("lag",lag("salary",1).over(windowSpec)).\
   withColumn("lead",lead("salary",1).over(windowSpec)) \
      .show()


+---------------+----------+------+----+----+
 EmployeeName|Department|Salary| lag|lead|
+---------------+----------+------+----+----+
 Matthew Kelly| Finance| 3000|null|4500|
 Andrea Lawson| Finance| 4500|3000|7500|
 Emily Wells| Finance| 7500|4500|null|
 Todd Stone| IT| 6885|null|7400|
 Terry Stokes| IT| 7400|6885|null|
 Hannah Weaver| Marketing| 5500|null|5500|
 Brian Chavez| Marketing| 5500|5500|8320|
 Dawn Aguirre| Marketing| 8320|5500|null|
 John Green| Sales| 4000|null|5000|
Stephanie Poole| Sales| 5000|4000|5000|
 Andrea Scott| Sales| 5000|5000|5601|
 David White| Sales| 5601|5000|6731|
 Erica Rush| Sales| 6731|5601|null|
+---------------+----------+------+----+----+

In [0]:
df.withColumn("percentage_rank",percent_rank().over(windowSpec)) \
    .show(truncate=False)

+---------------+----------+------+---------------+
EmployeeName |Department|Salary|percentage_rank|
+---------------+----------+------+---------------+
Matthew Kelly |Finance |3000 |0.0 |
Andrea Lawson |Finance |4500 |0.5 |
Emily Wells |Finance |7500 |1.0 |
Todd Stone |IT |6885 |0.0 |
Terry Stokes |IT |7400 |1.0 |
Hannah Weaver |Marketing |5500 |0.0 |
Brian Chavez |Marketing |5500 |0.0 |
Dawn Aguirre |Marketing |8320 |1.0 |
John Green |Sales |4000 |0.0 |
Stephanie Poole|Sales |5000 |0.25 |
Andrea Scott |Sales |5000 |0.25 |
David White |Sales |5601 |0.75 |
Erica Rush |Sales |6731 |1.0 |
+---------------+----------+------+---------------+

In [0]:
df.withColumn("lag",lag("salary",1).over(windowSpec)).\
   withColumn("lead",lead("salary",1).over(windowSpec)) \
      .show()

+---------------+----------+------+----+----+
 EmployeeName|Department|Salary| lag|lead|
+---------------+----------+------+----+----+
 Matthew Kelly| Finance| 3000|null|4500|
 Andrea Lawson| Finance| 4500|3000|7500|
 Emily Wells| Finance| 7500|4500|null|
 Todd Stone| IT| 6885|null|7400|
 Terry Stokes| IT| 7400|6885|null|
 Brian Chavez| Marketing| 5500|null|5500|
 Hannah Weaver| Marketing| 5500|5500|8320|
 Dawn Aguirre| Marketing| 8320|5500|null|
 John Green| Sales| 4000|null|5000|
 Andrea Scott| Sales| 5000|4000|5000|
Stephanie Poole| Sales| 5000|5000|5601|
 David White| Sales| 5601|5000|6731|
 Erica Rush| Sales| 6731|5601|null|
+---------------+----------+------+----+----+

In [0]:
from pyspark.sql.functions import lead   ,avg,min,max ,sum
df.withColumn("lead",lead("salary",1).over(windowSpec)) \
    .show()

+---------------+----------+------+----+
 EmployeeName|Department|Salary|lead|
+---------------+----------+------+----+
 Matthew Kelly| Finance| 3000|4500|
 Andrea Lawson| Finance| 4500|7500|
 Emily Wells| Finance| 7500|null|
 Todd Stone| IT| 6885|7400|
 Terry Stokes| IT| 7400|null|
 Hannah Weaver| Marketing| 5500|5500|
 Brian Chavez| Marketing| 5500|8320|
 Dawn Aguirre| Marketing| 8320|null|
 John Green| Sales| 4000|5000|
Stephanie Poole| Sales| 5000|5000|
 Andrea Scott| Sales| 5000|5601|
 David White| Sales| 5601|6731|
 Erica Rush| Sales| 6731|null|
+---------------+----------+------+----+

In [0]:
#Withour orderby in window function
df.withColumn("AvgSal",avg("salary").over(windowSpec_NOB)).withColumn("SumSal",sum(col("salary")).over(windowSpec_NOB)) \
    .show()

+---------------+----------+------+------+------+
 EmployeeName|Department|Salary|AvgSal|SumSal|
+---------------+----------+------+------+------+
 Matthew Kelly| Finance| 3000|5000.0| 15000|
 Andrea Lawson| Finance| 4500|5000.0| 15000|
 Emily Wells| Finance| 7500|5000.0| 15000|
 Terry Stokes| IT| 7400|7142.5| 14285|
 Todd Stone| IT| 6885|7142.5| 14285|
 Hannah Weaver| Marketing| 5500|6440.0| 19320|
 Dawn Aguirre| Marketing| 8320|6440.0| 19320|
 Brian Chavez| Marketing| 5500|6440.0| 19320|
Stephanie Poole| Sales| 5000|5266.4| 26332|
 Erica Rush| Sales| 6731|5266.4| 26332|
 David White| Sales| 5601|5266.4| 26332|
 John Green| Sales| 4000|5266.4| 26332|
 Andrea Scott| Sales| 5000|5266.4| 26332|
+---------------+----------+------+------+------+

In [0]:
#with order by in window function
df.withColumn("AvgSal",avg("salary").over(windowSpec)).withColumn("SumSal",sum(col("salary")).over(windowSpec)) \
    .show()

+---------------+----------+------+-----------------+------+
 EmployeeName|Department|Salary| AvgSal|SumSal|
+---------------+----------+------+-----------------+------+
 Matthew Kelly| Finance| 3000| 3000.0| 3000|
 Andrea Lawson| Finance| 4500| 3750.0| 7500|
 Emily Wells| Finance| 7500| 5000.0| 15000|
 Todd Stone| IT| 6885| 6885.0| 6885|
 Terry Stokes| IT| 7400| 7142.5| 14285|
 Brian Chavez| Marketing| 5500| 5500.0| 11000|
 Hannah Weaver| Marketing| 5500| 5500.0| 11000|
 Dawn Aguirre| Marketing| 8320| 6440.0| 19320|
 John Green| Sales| 4000| 4000.0| 4000|
 Andrea Scott| Sales| 5000|4666.666666666667| 14000|
Stephanie Poole| Sales| 5000|4666.666666666667| 14000|
 David White| Sales| 5601| 4900.25| 19601|
 Erica Rush| Sales| 6731| 5266.4| 26332|
+---------------+----------+------+-----------------+------+

In [0]:

windowSpec_unboundedpre = Window.partitionBy(col("Department")).orderBy("Salary").rowsBetween(Window.unboundedPreceding,Window.currentRow)
windowSpec_prevrow = Window.partitionBy(col("department")).orderBy("salary").rowsBetween(-1,Window.currentRow)
windowSpec_nextvrow = Window.partitionBy(col("department")).orderBy("salary").rowsBetween(Window.currentRow,1)

In [0]:
#Running totals per department
windowSpec_unboundedpre = Window.partitionBy(col("Department")).orderBy("Salary")\
.rowsBetween(Window.unboundedPreceding,Window.currentRow)

df.withColumn("AvgSal",avg("Salary").over(windowSpec_unboundedpre))\
.withColumn("SumSal",sum(col("Salary")).over(windowSpec_unboundedpre)) \
    .show()

+---------------+----------+------+-----------------+------+
 EmployeeName|Department|Salary| AvgSal|SumSal|
+---------------+----------+------+-----------------+------+
 Matthew Kelly| Finance| 3000| 3000.0| 3000|
 Andrea Lawson| Finance| 4500| 3750.0| 7500|
 Emily Wells| Finance| 7500| 5000.0| 15000|
 Todd Stone| IT| 6885| 6885.0| 6885|
 Terry Stokes| IT| 7400| 7142.5| 14285|
 Brian Chavez| Marketing| 5500| 5500.0| 5500|
 Hannah Weaver| Marketing| 5500| 5500.0| 11000|
 Dawn Aguirre| Marketing| 8320| 6440.0| 19320|
 John Green| Sales| 4000| 4000.0| 4000|
 Andrea Scott| Sales| 5000| 4500.0| 9000|
Stephanie Poole| Sales| 5000|4666.666666666667| 14000|
 David White| Sales| 5601| 4900.25| 19601|
 Erica Rush| Sales| 6731| 5266.4| 26332|
+---------------+----------+------+-----------------+------+

In [0]:
# Sum with next rows
df.withColumn("SumSal",sum(col("Salary")).over(windowSpec_nextvrow)) \
    .show(11)

+---------------+----------+------+------+
 EmployeeName|Department|Salary|SumSal|
+---------------+----------+------+------+
 Matthew Kelly| Finance| 3000| 7500|
 Andrea Lawson| Finance| 4500| 12000|
 Emily Wells| Finance| 7500| 7500|
 Todd Stone| IT| 6885| 14285|
 Terry Stokes| IT| 7400| 7400|
 Brian Chavez| Marketing| 5500| 11000|
 Hannah Weaver| Marketing| 5500| 13820|
 Dawn Aguirre| Marketing| 8320| 8320|
 John Green| Sales| 4000| 9000|
 Andrea Scott| Sales| 5000| 10000|
Stephanie Poole| Sales| 5000| 10601|
+---------------+----------+------+------+
only showing top 11 rows

In [0]:
#Sum with prev row
df.withColumn("SumSal",sum(col("Salary")).over(windowSpec_prevrow)) \
    .show()

+---------------+----------+------+------+
 EmployeeName|Department|Salary|SumSal|
+---------------+----------+------+------+
 Matthew Kelly| Finance| 3000| 3000|
 Andrea Lawson| Finance| 4500| 7500|
 Emily Wells| Finance| 7500| 12000|
 Todd Stone| IT| 6885| 6885|
 Terry Stokes| IT| 7400| 14285|
 Brian Chavez| Marketing| 5500| 5500|
 Hannah Weaver| Marketing| 5500| 11000|
 Dawn Aguirre| Marketing| 8320| 13820|
 John Green| Sales| 4000| 4000|
 Andrea Scott| Sales| 5000| 9000|
Stephanie Poole| Sales| 5000| 10000|
 David White| Sales| 5601| 10601|
 Erica Rush| Sales| 6731| 12332|
+---------------+----------+------+------+

In [0]:
#GroupBy and Agg 
df.selectExpr("cast(Salary as double) as Salary","Department")\
.groupBy('Department') \
.agg(sum('Salary').alias("SumofSalary"), avg('Salary').alias("AvgofSalary")\
     , max('Salary').alias("MaxofSalary") \
    ).show()

+----------+-----------+-----------+-----------+
Department|SumofSalary|AvgofSalary|MaxofSalary|
+----------+-----------+-----------+-----------+
 Sales| 26332.0| 5266.4| 6731.0|
 Finance| 15000.0| 5000.0| 7500.0|
 Marketing| 19320.0| 6440.0| 8320.0|
 IT| 14285.0| 7142.5| 7400.0|
+----------+-----------+-----------+-----------+

Testing UDF

In [0]:
from faker import Faker

faker = Faker()

In [0]:
from pyspark.sql.functions import row_number,rank,dense_rank,percent_rank,ntile,lead,lag,col,lit
from pyspark.sql.types import StructType,StructField, StringType, IntegerType


newempData = ((faker.name(), "Sales", faker.random_int(5000, 10000)), \
    (faker.name(), "Sales", faker.random_int(5000, 10000)),  \
    (faker.name(), "Sales", faker.random_int(5000, 10000)),   \
    (faker.name(), "Sales", faker.random_int(5000, 10000)),    \
    (faker.name(), "Sales", faker.random_int(5000, 10000)), \
    (faker.name(), "Finance", faker.random_int(5000, 10000)),  \
    (faker.name(), "Finance", faker.random_int(5000, 10000)),  \
    (faker.name(), "Finance", faker.random_int(5000, 10000)),    \
    (faker.name(), "Marketing", faker.random_int(5000, 10000)), \
    (faker.name(), "Marketing", faker.random_int(5000, 10000)),\
    (faker.name(), "IT", faker.random_int(5000, 10000)),\
    (faker.name(), "IT", faker.random_int(5000, 10000)),\
    (None, "Marketing", faker.random_int(5000, 10000)),\
    (faker.name(), "Marketing", faker.random_int(5000, 10000))\
  )

empSchema = StructType([ \
    StructField("EmployeeName",StringType(),True), \
    StructField("Department",StringType(),True), \
    StructField("Salary", IntegerType(), True) \
  ])
 

df_udf = spark.createDataFrame(data = newempData, schema = empSchema)


In [0]:
df_udf.printSchema()
df_udf.show(20)

root
-- EmployeeName: string (nullable = true)
-- Department: string (nullable = true)
-- Salary: integer (nullable = true)

+--------------------+----------+------+
 EmployeeName|Department|Salary|
+--------------------+----------+------+
 Loretta Mitchell| Sales| 7396|
 Deanna Hutchinson| Sales| 6916|
Mr. Spencer Gardn...| Sales| 5717|
 Lauren Chapman| Sales| 6045|
 Angela Lozano| Sales| 5245|
 Stacey Jackson| Finance| 5354|
 Jennifer Gray| Finance| 6148|
 James Cohen| Finance| 5368|
 Vanessa Phillips| Marketing| 7789|
 Ricardo Coffey| Marketing| 5319|
 Amber Spencer| IT| 9117|
 Andrew Bishop| IT| 7811|
 null| Marketing| 7591|
 Angela Adams| Marketing| 9217|
+--------------------+----------+------+

In [0]:
df_udf.createOrReplaceTempView("emp")

In [0]:
%sql
select * from emp where employeename is null

EmployeeName,Department,Salary
null,Marketing,7591


In [0]:
df_json=spark.read.option("multiline","true")\
.json("dbfs:/mnt/workshop/staging/jsondata/")

df_json.select ("Fuel","Transmission","about","cost").show()


+--------+------------+--------------------+---------+
 Fuel|Transmission| about| cost|
+--------+------------+--------------------+---------+
Electric| automatic|Labore officia in...|$7,543.34|
 diesel| automatic|Proident labore o...|$8,675.48|
Electric| automatic|Irure qui ipsum e...|$7,022.03|
Electric| manual|Qui nostrud occae...|$5,691.45|
 petrol| manual|Ex occaecat non m...|$7,354.97|
Electric| manual|Mollit fugiat lab...|$8,251.95|
 diesel| automatic|Consequat occaeca...|$2,865.07|
Electric| manual|Nostrud est cillu...|$6,564.90|
 petrol| automatic|Cillum Lorem exer...|$6,263.10|
Electric| manual|Eu aute officia a...|$5,400.73|
Electric| automatic|Dolore ullamco en...|$3,860.15|
 diesel| automatic|Anim nisi occaeca...|$7,109.39|
 diesel| manual|Culpa velit incid...|$9,199.09|
 petrol| automatic|Quis enim reprehe...|$4,598.71|
 diesel| automatic|Non enim non magn...|$4,700.08|
Electric| manual|In pariatur enim ...|$9,552.65|
 diesel| manual|Excepteur est iru...|$2,932.99|
 petrol| automatic|Anim ea id veniam...|$7,186.09|
 petrol| automatic|Cillum velit labo...|$8,811.24|
 diesel| manual|Cupidatat laboris...|$4,695.38|
+--------+------------+--------------------+---------+
only showing top 20 rows

In [0]:
from pyspark.sql.types import *

In [0]:
cust_schema = StructType([
    StructField("C_CUSTKEY", IntegerType()),
    StructField("C_NAME", StringType()),
    StructField("C_ADDRESS", StringType()),
    StructField("C_NATIONKEY", ShortType()),
    StructField("C_PHONE", StringType()),
    StructField("C_ACCTBAL", DoubleType()),
    StructField("C_MKTSEGMENT", StringType()),
    StructField("C_COMMENT", StringType())
])

df_cust= spark.read.option("header", True).option("field_delimiter", ",")\
.schema(cust_schema).csv("dbfs:/mnt/workshop/staging/customer/csvFiles/")
df_cust.show()


+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
C_CUSTKEY| C_NAME| C_ADDRESS|C_NATIONKEY| C_PHONE|C_ACCTBAL|C_MKTSEGMENT| C_COMMENT|
+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
 35165|Customer#000035165| eNQSvDTld1 f7JmY| 0|10-173-541-5438| 4767.46| AUTOMOBILE|special excuses. ...|
 30597|Customer#000030597| S9s1dDut8Q| 0|10-607-243-5581| -639.62| FURNITURE|lithely ruthless ...|
 42279|Customer#000042279| ABcVdNnA3JFB7bK5| 0|10-934-981-2863| 2236.39| MACHINERY|the even deposits...|
 42578|Customer#000042578|l6VNaE7iSZFtkSC5f...| 0|10-281-998-8028| 6429.8| BUILDING|y alongside of th...|
 37854|Customer#000037854| dL6LCTLpY9hjLTrZ7g| 0|10-909-820-4270| 9549.78| BUILDING|inder blithely de...|
 40053|Customer#000040053| qh8Q6gaffF73cm73K2R| 0|10-593-423-2533| 209.4| MACHINERY|c pinto beans. sp...|
 44060|Customer#000044060|XXYMZ4Jd4PY3WJZA5...| 0|10-519-920-9801| 1150.68| BUILDING|according to the ...|
 49988|Customer#000049988|oYYSmhl,K3t AwKzm...| 0|10-704-487-3360| 3708.73| BUILDING|t quickly. pendin...|
 43569|Customer#000043569|ANmTNESWDI17e2pG7...| 0|10-911-273-3629| 3239.41| MACHINERY|of the slyly even...|
 27443|Customer#000027443|xOgk0us699smqWP3U...| 0|10-492-101-8357| 4304.73| MACHINERY|ly bold accounts ...|
 36438|Customer#000036438|OMSO,p4PXeCsGd47b...| 0|10-757-611-4797| 6332.86| HOUSEHOLD|blithely express ...|
 51121|Customer#000051121| XbffpKB4tLLwn| 0|10-389-120-8360| 5667.19| FURNITURE|ccounts are blith...|
 34128|Customer#000034128| ,kvcrAtXRHiTV,gzug| 0|10-926-376-8200| 7709.0| BUILDING|quickly at the ex...|
 40615|Customer#000040615| bJxfUzUxws| 0|10-116-341-7891| 1798.06| FURNITURE|ular requests int...|
 42757|Customer#000042757|mA9pHU8Gz0iGClojK...| 0|10-697-130-7058| -725.18| BUILDING|ly express ideas....|
 47191|Customer#000047191|DpltVI8VuIsW2Qffj...| 0|10-337-483-4822| 554.21| HOUSEHOLD|boost according t...|
 27770|Customer#000027770| skRhheAKfBp7koLdmwa| 0|10-508-645-6474| -425.75| BUILDING|theodolites. quic...|
 37421|Customer#000037421|OtpgofCsfVl83zJ0v...| 0|10-361-118-7857| 6544.22| BUILDING|brave foxes. care...|
 37680|Customer#000037680|j0Ris2M4 o1L vE9A...| 0|10-417-928-9540| 3694.18| FURNITURE|s. carefully iron...|
 32646|Customer#000032646|vXEIPLpay1O8HpNTE...| 0|10-262-541-2360| 7785.48| AUTOMOBILE|eep according to ...|
+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
only showing top 20 rows

In [0]:
df_cust.write.mode("overwrite")\
.parquet('dbfs:/mnt/workshop/staging/customer/newsparkParquetFiles/')

In [0]:

df_cust.write.partitionBy("C_MKTSEGMENT","C_NATIONKEY").mode("overwrite")\
.parquet('dbfs:/mnt/workshop/staging/customer/newsparkpartitionParquetFiles/')

In [0]:
df_cust.write.format("delta").mode("append").partitionBy("C_MKTSEGMENT","C_NATIONKEY")\
.saveAsTable("NewCustomer") 



In [0]:
spark.sql("select * from NewCustomer").show(5)

+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
C_CUSTKEY| C_NAME| C_ADDRESS|C_NATIONKEY| C_PHONE|C_ACCTBAL|C_MKTSEGMENT| C_COMMENT|
+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
 35165|Customer#000035165| eNQSvDTld1 f7JmY| 0|10-173-541-5438| 4767.46| AUTOMOBILE|special excuses. ...|
 32646|Customer#000032646|vXEIPLpay1O8HpNTE...| 0|10-262-541-2360| 7785.48| AUTOMOBILE|eep according to ...|
 39427|Customer#000039427|84Z3GWAHvvhBpSXXO...| 0|10-583-834-1564| 2504.65| AUTOMOBILE|mpress carefully ...|
 36996|Customer#000036996|GeKaki PTfhgyPiG3...| 0|10-729-794-1813| 6672.41| AUTOMOBILE|ilent instruction...|
 49976|Customer#000049976| SiId7JMKMwGzBUC| 0|10-916-468-8977| 9233.09| AUTOMOBILE|ronic ideas cajol...|
+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
only showing top 5 rows